<a href="https://colab.research.google.com/github/PhaseYon/AI-Flashcards/blob/main/AI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch pandas

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
import pandas as pd

In [ ]:
# Load the TSV dataset
df = pd.read_csv('/content/profpho.tsv', sep='\t', quotechar='"')

# Preview your data
df.head()


,question,answer
0,What is professional real estate photography?,Professional real estate photography captures ...
1,Why should I use professional photography for ...,Professional photography increases online rese...
2,How many photos do I receive with the interior...,You will receive at least 12 high-resolution m...
3,Do you offer retouching or photo enhancement?,"Yes, basic retouching is included, such as gra..."
4,How long does it take to receive the final pho...,You will receive your digital photos within 24...


In [ ]:
# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the questions and answers (treat answers as labels)
def tokenize_function_with_labels(examples):
    # Tokenize the question
    question_encodings = tokenizer(examples['question'], padding="max_length", truncation=True, max_length=512)

    # Tokenize the answer (this will be used as the label)
    answer_encodings = tokenizer(examples['answer'], padding="max_length", truncation=True, max_length=512)

    # Return the tokenized questions and answers as labels
    return { **question_encodings, "labels": answer_encodings['input_ids'] }
from datasets import Dataset
# Convert your pandas dataframe to Huggingface dataset format
dataset = Dataset.from_pandas(df)

# Apply the tokenizer with labels to the dataset
dataset = dataset.map(tokenize_function_with_labels, batched=True)

# Format the dataset for PyTorch
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Preview the dataset structure
dataset[0]  # Check the first example to ensure it looks correct


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

{'input_ids': tensor([ 101, 2054, 2003, 2658, 2613, 3776, 5855, 1029,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Use a model like DistilBERT for classification or DistilBART/T5 for sequence-to-sequence
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Small batch size
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Mixed precision for better memory management
    disable_tqdm=True
)

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

# Define evaluation metric
def compute_metrics(p):
    pred, labels = p
    pred_ids = torch.argmax(pred, dim=-1)
    accuracy = (pred_ids == labels).mean().item()
    return {'accuracy': accuracy}

# Initialize Trainer
trainer = Trainer(
    model=model,                       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                # training arguments, defined above
    train_dataset=dataset,             # training dataset
    eval_dataset=dataset,              # evaluation dataset
    compute_metrics=compute_metrics    # evaluation metrics
)

In [ ]:
trainer.train()

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not tuple